In [1]:
import os
import networkx as nx
import pandas as pd
import spacy

In [2]:
nlp = spacy.load("fr_core_news_sm")

In [3]:
def extract_entities(text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents if ent.label_ == "PER"]

In [4]:
# Fonction pour créer le graphe à partir d'un texte
def create_graph(text):
    G = nx.Graph()
    entities = extract_entities(text)

    for i in range(len(entities)):
        for j in range(i + 1, len(entities)):
            # Calculer le nombre de mots entre les entités, limité à 15
            words_between = min(15, abs(entities.index(entities[j]) - entities.index(entities[i])) - 1)
            if words_between > 0:
                # Ajouter l'arête entre les entités avec le poids (nombre de mots entre elles)
                G.add_edge(entities[i], entities[j], weight=words_between)

    return G


In [5]:
# Liste de chapitres avec les codes de livre associés
books = [
    (list(range(1, 20)), "paf"),
    (list(range(1, 19)), "lca"),
]

In [6]:
# Initialiser le dictionnaire pour le DataFrame
df_dict = {"ID": [], "graphml": []}


In [7]:
# Chemins vers les dossiers contenant les chapitres
folder_paths = {
    "paf": "../data/corpus_ASIMOV_leaderboard/corpus_asimov_leaderboard/prelude_a_fondation/",
    "lca": "../data/corpus_ASIMOV_leaderboard/corpus_asimov_leaderboard/les_cavernes_d_acier/",
}


In [8]:
# Initialiser le dictionnaire pour le DataFrame
df_dict = {"ID": [], "graphml": []}

# Boucle à travers les livres et les chapitres
for chapters, book_code in books:
    for chapter in chapters:
        # Lire le texte du chapitre à partir du fichier
        file_path = os.path.join(folder_paths[book_code], f"chapter_{chapter}.txt.preprocessed")
        with open(file_path, "r", encoding="utf-8") as file:
            chapter_text = file.read()

        # Créer le graphe à partir du texte du chapitre
        G = create_graph(chapter_text)

        # Ajouter les attributs "names" aux nœuds
        for node in G.nodes:
            G.nodes[node]["names"] = node

        # Ajouter les données au dictionnaire
        df_dict["ID"].append(f"{book_code}{chapter}")
        df_dict["graphml"].append("".join(nx.generate_graphml(G)))


In [9]:
# Créer le DataFrame
df = pd.DataFrame(df_dict)
df["graphml"] = df["graphml"].apply(lambda x: f'"{x}"') 

# Exporter le DataFrame vers un fichier CSV
df.to_csv("submission_Niang_Essabri.csv", index=False)

In [11]:
import os
import networkx as nx
import pandas as pd
import spacy

# Charger le modèle de langue SpaCy
nlp = spacy.load("fr_core_news_sm")

# Fonction pour extraire les entités nommées (personnages) d'un texte
def extract_entities(text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents if ent.label_ == "PER"]

# Fonction pour créer le graphe à partir d'un texte
def create_graph(text):
    G = nx.Graph()
    entities = extract_entities(text)

    for i in range(len(entities)):
        for j in range(i + 1, len(entities)):
            # Calculer le nombre de mots entre les entités, limité à 15
            words_between = min(15, abs(entities.index(entities[j]) - entities.index(entities[i])) - 1)
            if words_between > 0:
                # Ajouter l'arête entre les entités avec le poids (nombre de mots entre elles)
                G.add_edge(entities[i], entities[j], weight=words_between)

    return G

# Chemins vers les dossiers contenant les chapitres
folder_paths = {
    "paf": "../data/corpus_ASIMOV_leaderboard/corpus_asimov_leaderboard/prelude_a_fondation/",
    "lca": "../data/corpus_ASIMOV_leaderboard/corpus_asimov_leaderboard/les_cavernes_d_acier/",
}

# Liste de chapitres avec les codes de livre associés
books = [
    (list(range(1, 20)), "paf"),  # Ajustement pour commencer à 0 et inclure le chapitre 18 pour "paf"
    (list(range(1, 19)), "lca"),  # Ajustement pour commencer à 0 et inclure le chapitre 17 pour "lca"
]

# Initialiser le dictionnaire pour le DataFrame
df_dict = {"ID": [], "graphml": []}

# Boucle à travers les livres et les chapitres
for chapters, book_code in books:
    for chapter in chapters:
        # Lire le texte du chapitre à partir du fichier
        file_path = os.path.join(folder_paths[book_code], f"chapter_{chapter}.txt.preprocessed")
        with open(file_path, "r", encoding="utf-8") as file:
            chapter_text = file.read()

        # Créer le graphe à partir du texte du chapitre
        G = create_graph(chapter_text)

        # Ajouter les attributs "names" aux nœuds
        for node in G.nodes:
            G.nodes[node]["names"] = node

        # Ajouter les données au dictionnaire
        graphml_content = "".join(nx.generate_graphml(G))
        df_dict["ID"].append(f"{book_code}{chapter}")
        df_dict["graphml"].append(f'"{graphml_content}"')  # Encadrer les valeurs de graphml avec des guillemets

# Créer le DataFrame
df = pd.DataFrame(df_dict)

# Exporter le DataFrame vers un fichier CSV
df.to_csv("submission_Niang_Essabri.csv", index=False)
